In [14]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Debido a que necesitamos que existan varios agentes por celda, elegimos ''MultiGrid''.
from mesa.space import MultiGrid

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import SimultaneousActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd
import random

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [15]:
"""Grid del modelo, contiene los estados de los agentes"""
def get_warehouse(model):
    grid = np.zeros( (model.grid.width, model.grid.height) )
    for (content, x, y) in model.grid.coord_iter():
        for cont in content:
            #asignamos el color a la celda de nuestra matriz segun lo que esta contenga
            if isinstance(cont, AgentRobot):
                if cont.state == 'no_box':
                    grid[x][y] = 1
                else:
                    grid[x][y] = 0.85
            else:
                if cont.state == 'box':
                    if cont.weight ==  1:
                        grid[x][y] = 0.20
                    if cont.weight ==  2:
                        grid[x][y] = 0.35
                    if cont.weight ==  3:
                        grid[x][y] = 0.50
                    if cont.weight ==  4:
                        grid[x][y] = 0.65
                    if cont.weight ==  5:
                        grid[x][y] = 0.80
                else:
                    grid[x][y] = 0
                
    return grid

In [16]:
"""Agente de las cajas"""
class AgentBox(Agent):
    def __init__(self, pos, model, state):
        super().__init__(pos, model)
        self.x, self.y = pos
        self.state = state
        self.weight = 1

In [17]:
"""Agente de los robots"""
class AgentRobot(Agent):
    def __init__(self, pos, model, height, width):
        super().__init__(pos, model)
        self.x, self.y = pos
        self.new_pos = None
        self.state = 'no_box'
        self.height = height
        self.width = width
        
    def step(self):
        flag = False
        if self.state == 'no_box':
            
            if self.model.count_box != self.model.num_box:
                self.model.steps += 1
                
            neighbours = self.model.grid.get_neighbors(self.pos, moore = True, include_center = True)
            for neighbour in neighbours:
                if (self.pos == neighbour.pos and 
                isinstance(neighbour, AgentBox) and 
                neighbour.state == 'box' and 
                neighbour.weight == 1):
                    self.state = 'with_box'
                    neighbour.state = 'empty'
                    flag = True
                    """AQUI CONTADOR DE CAJAS, CUANDO LLEGUE AL MISMO NUMERO DE CAJAS BREAK (opcion 1)"""
                    self.model.count_box += 1
                    break
                
            if flag == False:
                n = random.choice([1, 2, 3, 4])
                if n == 1:
                    self.x += 1
                elif n == 2:
                    self.x -= 1
                elif n == 3:
                    self.y += 1
                else:
                    self.y -= 1
                        
        else:
            
            if self.model.count_box != self.model.num_box - 1:
                self.model.steps += 1
                
            if self.x > self.model.pile_full:
                self.x -= 1
            
            elif self.x < self.model.pile_full:
                self.x += 1
                
            else:
                if self.y == self.model.piles:
                    neighbours = self.model.grid.get_neighbors(self.pos, moore = True, include_center = True)
                    for neighbour in neighbours:
                        if self.pos == neighbour.pos and isinstance(neighbour, AgentBox):
                            self.state = 'no_box'
                            """AQUI CONTADOR DE CAJAS, CUANDO LLEGUE AL MISMO NUMERO DE CAJAS BREAK (opcion 2)"""
                            self.x += 1
                            if neighbour.state == 'empty':
                                neighbour.state = 'box'
                                neighbour.weight = 1
                            else:
                                if neighbour.weight < 4 and neighbour.weight > 0:
                                    neighbour.weight += 1
                                elif neighbour.weight == 4:
                                    neighbour.weight += 1
                                    self.model.piles += 1
                                    if self.model.piles == self.width:
                                        self.model.pile_full += 1
                                        self.model.piles = 0
                                        
                elif self.model.piles < self.y:
                    self.y -= 1
                  
                else:
                    self.y += 1
                    
            
        self.new_pos = self.x, self.y
        self.model.grid.move_agent(self, self.new_pos)
        

In [18]:
"""Modelo de los agentes"""
class Warehouse(Model):

    def __init__(self, height, width, num_box):
        self.num_robots = 5
        self.num_box = num_box
        self.grid = MultiGrid(height, width, True)
        self.piles = 0
        self.pile_full = 0
        self.end = 0
        self.schedule = SimultaneousActivation(self)
        
        self.steps = 0
        self.count_box = 0

        # Celdas con robots aleatorias
        empty_cells = list(self.grid.empties)
        for i in range(5):
            emptyCell = random.choice(empty_cells)
            robot = AgentRobot(emptyCell, self, height, width)
            self.grid.place_agent(robot, emptyCell)
            self.schedule.add(robot)
            empty_cells.remove(emptyCell)
            
        # Celdas con cajas aleatorias   
        empty_cells = list(self.grid.empties)    
        for cells in range(num_box):
            emptyCell = random.choice(empty_cells)
            box = AgentBox(emptyCell, self, 'box')
            self.grid.place_agent(box, emptyCell)
            self.schedule.add(box)
            empty_cells.remove(emptyCell)
            
        # Celdas restantes vacías
        empty_cells = list(self.grid.empties)
        for cells in empty_cells:
            box = AgentBox(cells, self, 'empty')
            self.grid.place_agent(box, cells)
            self.schedule.add(box)
            
           
        self.datacollector = DataCollector(model_reporters={'Warehouse': get_warehouse})
    
    
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
        

In [63]:
#Inicializar variables

HEIGHT = 10
WIDTH = 10
number_of_boxes = 30

"""Modelo con límite de steps"""
#max_steps = 300
#i = 0
#model = Warehouse(HEIGHT, WIDTH, number_of_boxes)
#while(i < max_steps):
#    model.step()
#    i += 1

"""Modelo con límite de tiempo"""
MAX_GENERATIONS = .1
start_time = time.time()
model = Warehouse(HEIGHT, WIDTH, number_of_boxes)

while((time.time() - start_time) < MAX_GENERATIONS):
    model.step()
    if(model.count_box < number_of_boxes):
        tiempo_total = time.time() - start_time
        
real_time = str(datetime.timedelta(seconds = (time.time() - start_time)))

In [64]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [65]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='RdPu')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [66]:
print("--------------------------------RESULTADOS---------------------------------")
print("Tiempo necesario hasta que todas las cajas estén apiladas", tiempo_total )
print("Numero de movimientos realizados por todos los robots", model.steps)
print("Numero de movimientos realizados por robot", model.steps//5)
anim

--------------------------------RESULTADOS---------------------------------
Tiempo necesario hasta que todas las cajas estén apiladas 0.03889346122741699
Numero de movimientos realizados por todos los robots 3486
Numero de movimientos realizados por robot 697
